In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [101 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14 http://ppa.launc

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-13 16:37:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-11-13 16:37:08 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M20-FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://cryptobucket606.s3.us-west-2.amazonaws.com/assetdetails.csv"
spark.sparkContext.addFile(url)
asset_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
asset_df.show()

+--------+------------------+----------------+
|Asset_ID|            Weight|      Asset_Name|
+--------+------------------+----------------+
|       2|2.3978952727983707|    Bitcoin Cash|
|       0|  4.30406509320417|    Binance Coin|
|       1| 6.779921907472252|         Bitcoin|
|       5|1.3862943611198906|          EOS.IO|
|       7|2.0794415416798357|Ethereum Classic|
|       6|5.8944028342648505|        Ethereum|
|       9|2.3978952727983707|        Litecoin|
|      11|1.6094379124341003|          Monero|
|      13| 1.791759469228055|            TRON|
|      12|2.0794415416798357|         Stellar|
|       3| 4.406719247264253|         Cardano|
|       8|1.0986122886681098|            IOTA|
|      10|1.0986122886681098|           Maker|
|       4|3.5553480614894135|        Dogecoin|
+--------+------------------+----------------+



In [ ]:
# Read in the Review dataset as a DataFrame
asset_df.printSchema()

root
 |-- Asset_ID: integer (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Asset_Name: string (nullable = true)



In [ ]:
url = "https://cryptobucket606.s3.us-west-2.amazonaws.com/train.csv"
spark.sparkContext.addFile(url)
train_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
train_df.show()

+----------+--------+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------------+
| timestamp|Asset_ID|Count|              Open|   High|               Low|             Close|            Volume|              VWAP|              Target|
+----------+--------+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------------+
|1514764860|       2| 40.0|           2376.58| 2399.5|           2357.14|           2374.59|       19.23300519|2373.1163915061647|-0.00421815238742...|
|1514764860|       0|  5.0|              8.53|   8.53|              8.53|              8.53|             78.38|              8.53|-0.01439896646896...|
|1514764860|       1|229.0|         13835.194|14013.8|          13666.11|         13850.176|       31.55006152|13827.062092689883|-0.01464322435573...|
|1514764860|       5| 32.0|            7.6596| 7.6596|            7.6567|            7.6

In [ ]:
train_df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- Asset_ID: string (nullable = true)
 |-- Count: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- VWAP: string (nullable = true)
 |-- Target: double (nullable = true)



In [ ]:
train_df.show()

+----------+--------+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------------+
| timestamp|Asset_ID|Count|              Open|   High|               Low|             Close|            Volume|              VWAP|              Target|
+----------+--------+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------------+
|1514764860|       2| 40.0|           2376.58| 2399.5|           2357.14|           2374.59|       19.23300519|2373.1163915061647|-0.00421815238742...|
|1514764860|       0|  5.0|              8.53|   8.53|              8.53|              8.53|             78.38|              8.53|-0.01439896646896...|
|1514764860|       1|229.0|         13835.194|14013.8|          13666.11|         13850.176|       31.55006152|13827.062092689883|-0.01464322435573...|
|1514764860|       5| 32.0|            7.6596| 7.6596|            7.6567|            7.6

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://cryptodata.cvzpfgwkwqa1.us-west-2.rds.amazonaws.com:5432/CryptoData"
config = {"user":"postgres", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write asset_df to table in RDS
asset_df.write.jdbc(url=jdbc_url, table='asset_details', mode=mode, properties=config)

In [ ]:
# Write asset_df to table in RDS
asset_df.write.jdbc(url=jdbc_url, table='train', mode=mode, properties=config)

Column<'CAST(from_unixtime((timestamp / 1000), yyyy-MM-dd HH:mm:ss) AS DATE)'>